In [7]:
import numpy as np
import pandas as pd
from warnings import filterwarnings
filterwarnings("ignore")
import os
print("Orig Dir:",os.getcwd())
os.chdir('../../')
print("Current Dir:",os.getcwd())

Orig Dir: E:\Pascal\DSR\drone_steering\models\playground
Current Dir: E:\Pascal\DSR\drone_steering


In [8]:
framebased_labels_dir = 'data/gesture/labels_framebased/'
print(os.listdir(framebased_labels_dir)[0:4])
features_dir = 'data/gesture/features/'
print(os.listdir(features_dir)[0:4])
file_features = 'features_flip_c_01_120.csv'
file_labels = 'labels_flip_c_01.csv'

data = pd.read_csv(features_dir + file_features)
labels = pd.read_csv(framebased_labels_dir + file_labels)
data.head()

['labels_flip_c_01.csv', 'labels_flip_c_02.csv', 'labels_flip_c_03.csv', 'labels_flip_l_01.csv']
['features_flip_c_01_120.csv', 'features_flip_c_02_120.csv', 'features_flip_c_03_120.csv', 'features_flip_l_01_120.csv']


,leftElbow_x,leftElbow_y,leftHip_x,leftHip_y,leftShoulder_x,leftShoulder_y,leftWrist_x,leftWrist_y,ms_since_last_frame,ms_since_start,rightElbow_x,rightElbow_y,rightHip_x,rightHip_y,rightShoulder_x,rightShoulder_y,rightWrist_x,rightWrist_y
0,330.652879,198.063794,317.111272,247.594685,318.645721,141.643284,338.098494,250.403484,0,0.0,244.917285,202.445251,275.317313,247.668226,252.832705,138.901809,247.234862,250.259751
1,330.654097,193.589369,317.292876,247.826779,319.605955,141.606901,339.914686,250.056719,107,107.0,246.100778,201.271757,276.424059,248.102428,253.846423,137.679758,248.110395,252.987147
2,331.448529,197.024456,317.852591,247.613809,319.906014,140.208933,340.343497,249.927276,121,228.0,244.875221,201.363363,273.596084,247.506937,255.344219,137.241180,248.820809,251.952821
3,331.817814,197.920270,318.037202,247.849019,319.819313,140.085572,340.734974,250.452507,116,344.0,245.576020,201.978553,273.625880,249.090494,255.177429,138.182819,248.975858,252.808843
4,331.810191,192.195117,318.051376,248.002438,320.712613,140.844489,341.786785,250.964349,111,455.0,245.073013,202.078767,273.534740,248.624276,255.220840,138.375098,248.799108,251.982110


In [10]:
from app_local.module import DataFrameInterpolator

In [15]:
def interpolate_and_generate_new_framebased_labels(data_orig, labels_orig, new_frmlen):
    
    temp_labels = labels_orig.copy()
    
    start_times = []
    end_times = []

    for j in range(labels_orig.shape[0]):
        start_times.append(data_orig.iloc[int(labels_orig.iloc[j]["real_start"])]["ms_since_start"])
        end_times.append(data_orig.iloc[int(labels_orig.iloc[j]["real_end"])]["ms_since_start"])
    
    temp_labels["start_time"] = start_times
    temp_labels["end_time"] = end_times
    
    di = DataFrameInterpolator()
    
    new_data = di.get_new_df(
        df = data_orig,
        frmlen = new_frmlen
    )
    
      
    j = 0
    has_start = False
    new_start_idx = []
    new_end_idx = []

    for i in range(new_data.shape[0]):
    
        t = new_data.iloc[i]["ms_since_start"]
        current_end = temp_labels.iloc[j]["end_time"]

        if has_start:
            if t > current_end - new_frmlen/2:
                new_end_idx.append(i)
                j = j+1
                has_start = False
                if j >= temp_labels.shape[0]:
                    break

        current_start = temp_labels.iloc[j]["start_time"]


        if not has_start:
            if t >= current_start - new_frmlen/2:
                new_start_idx.append(i)
                has_start = True
            

    temp_labels["new_idx_start"] = new_start_idx
    temp_labels["new_idx_end"] = new_end_idx
    
    temp_labels["new_start_time"] = list(new_data.loc[new_start_idx,"ms_since_start"])
    temp_labels["new_end_time"] = list(new_data.loc[new_end_idx,"ms_since_start"])
    
    return new_data, temp_labels

In [18]:
d,l = interpolate_and_generate_new_framebased_labels(data, labels, 100)

In [21]:
d[["ms_since_start"]]
l

,real_start,real_end,label,start_time,end_time,new_idx_start,new_idx_end,new_start_time,new_end_time
0,41,45,3,4283.0,4787.0,43,48,4300,4800
1,87,91,3,9160.0,9593.0,92,96,9200,9600
2,129,132,3,13526.0,13831.0,135,138,13500,13800
3,170,173,3,17802.0,18107.0,178,181,17800,18100
4,188,193,3,19657.0,20157.0,197,202,19700,20200
5,206,209,3,21522.0,21822.0,215,218,21500,21800
6,252,253,3,26270.0,26369.0,263,264,26300,26400
7,293,295,3,30487.0,30689.0,305,307,30500,30700
8,346,346,3,35962.0,35962.0,360,361,36000,36100
9,391,395,3,40704.0,41160.0,407,412,40700,41200


In [ ]:
labels

In [ ]:
start_times = []
end_times = []

for j in range(labels.shape[0]):
    start_times.append(data.iloc[int(labels.iloc[j]["real_start"])]["ms_since_start"])
    end_times.append(data.iloc[int(labels.iloc[j]["real_end"])]["ms_since_start"])
    
labels["start_time"] = start_times
labels["end_time"] = end_times

In [ ]:
di = DataFrameInterpolator()
frmlen = 50
new_df = di.get_new_df(df=data,frmlen=frmlen)
new_df.tail()

In [ ]:
j = 0
has_start = False
new_start_idx = []
new_end_idx = []

for i in range(new_df.shape[0]):
    
    t = new_df.iloc[i]["ms_since_start"]
    current_end = labels.iloc[j]["end_time"]
    
    if has_start:
        if t > current_end - frmlen/2:
            new_end_idx.append(i)
            j = j+1
            has_start = False
            if j >= labels.shape[0]:
                break
    
    current_start = labels.iloc[j]["start_time"]
    
        
    if not has_start:
        if t >= current_start - frmlen/2:
            new_start_idx.append(i)
            has_start = True
            
    #print("i:",i, "j:",j, has_start)
    #print("t:", t, "s:", current_start, "e:", current_end)
    #print("")
            
labels["new_idx_start"] = new_start_idx
labels["new_idx_end"] = new_end_idx
labels

In [ ]:
list(new_df.loc[new_start_idx,"ms_since_start"])


In [ ]:
labels["new_start_time"] = list(new_df.loc[new_start_idx,"ms_since_start"])
labels["new_end_time"] = list(new_df.loc[new_end_idx,"ms_since_start"])
labels